In [105]:
import pandas as pd
import tensorflow as tf
print (tf.__version__)
column=["a","b","c","d","t"]
#column=["花萼长度","花萼宽度","花瓣长度","花瓣宽度","花的种类"]
data_train=pd.read_csv('alldata/iris_training.csv',names=column,header=0)
data_test=pd.read_csv('alldata/iris_test.csv',names=column,header=0)
print data_train.shape
print data_test.shape
data_train.head()

1.10.1
(120, 5)
(30, 5)


,a,b,c,d,t
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [89]:
train_x, train_y = data_train, data_train.pop('t')
test_x, test_y = data_test, data_test.pop('t')
train_x.shape
train_y.head()
#train_x.head()

0    2
1    1
2    2
3    0
4    0
Name: t, dtype: int64

In [90]:
my_feature_columns=[]
for key in train_x.keys():
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[_NumericColumn(key='a', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='b', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='c', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='d', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [91]:
classifier = tf.estimator.DNNClassifier(
    # 这个模型接受哪些输入的特征
    feature_columns=my_feature_columns,
    # 包含两个隐藏层，每个隐藏层包含10个神经元.
    hidden_units=[10, 10],
    # 最终结果要分成的几类
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f497afc62d0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmp3sxmlI', '_train_distribute': None, '_save_summary_steps': 100}


In [141]:
def train_func(train_x,train_y):
    dataset=tf.data.Dataset.from_tensor_slices((dict(train_x), train_y))
    dataset = dataset.shuffle(200).repeat().batch(1)
    return dataset

In [142]:
classifier.train(input_fn=lambda:train_func(train_x,train_y),steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp3sxmlI/model.ckpt-13000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 13000 into /tmp/tmp3sxmlI/model.ckpt.
INFO:tensorflow:loss = 0.0040664137, step = 13000
INFO:tensorflow:global_step/sec: 662.445
INFO:tensorflow:loss = 0.005831965, step = 13100 (0.153 sec)
INFO:tensorflow:global_step/sec: 877.301
INFO:tensorflow:loss = 0.00026008085, step = 13200 (0.114 sec)
INFO:tensorflow:global_step/sec: 854.394
INFO:tensorflow:loss = 0.0004749362, step = 13300 (0.117 sec)
INFO:tensorflow:global_step/sec: 825.559
INFO:tensorflow:loss = 0.00013386307, step = 13400 (0.121 sec)
INFO:tensorflow:global_step/sec: 803.645
INFO:tensorflow:loss = 0.17415029, step = 13500 (0.124 sec)
INFO:tensorflow:global_step/sec: 788.668
INFO:te

In [143]:
def eval_input_fn(features, labels, batch_size):
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
 
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    return dataset

In [146]:
from __future__ import division
predict_arr = []
predictions = classifier.predict(
        input_fn=lambda:eval_input_fn(test_x,labels=test_y,batch_size=100))
for predict in predictions:
    predict_arr.append(predict['probabilities'].argmax())
result = predict_arr == test_y
result1 = [w for w in result if w == True]
print("准确率为 %s"%str((len(result1)/len(result)*100)))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp3sxmlI/model.ckpt-14000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
准确率为 96.6666666667
